In [1]:
!pip install gensim 

  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110324 sha256=df679d674a551b9f8023204773254cbba3cd6558c4b7d8f53241fb0ea50c0e47
  Stored in directory: c:\users\manish\appdata\local\pip\cache\wheels\56\b5\6d\86dbe4f29d4688e5163a8b8c6b740494310040286fca4dc648
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [1]:
import gensim.downloader as api

In [2]:
word_vectors = api.load("glove-wiki-gigaword-100")

In [74]:
vector = word_vectors["cisco"]
print(vector,vector.shape)

[ 0.49801   0.39082   0.25831   0.17577   0.38281  -0.91952  -0.0208
 -0.6191   -0.3046   -0.60664   0.73944  -0.60114   0.20317  -0.92488
 -0.08956  -0.5604    0.36087   0.48024   0.17788   0.86619  -0.73105
 -0.50023   0.021111  1.8978   -0.27749   0.13275   0.56186   0.36651
 -0.65007   0.28827   0.58561   0.67113  -0.63215  -0.10346   0.48538
  0.5028   -0.24947  -0.2718    1.2757   -0.38339   0.13535  -0.10458
  0.19979  -0.42208  -1.1576    0.68574   0.11166   0.1972    0.53462
 -0.51698  -0.31698   0.25235  -0.40096  -0.17295  -0.43887  -0.65641
  0.15099   0.1673    0.78664   0.19512  -0.38113  -0.50796  -0.29866
  0.1746   -1.4846   -0.65841   0.53127  -0.067229  0.7765    0.41547
  0.18037   0.64929  -0.2771   -0.16704  -0.5604   -0.4464    0.31273
  0.21662  -0.045884  0.6484    0.98131  -0.45795  -0.91943   0.33496
 -0.85287  -0.47377   0.71127  -0.1464    0.42078   0.39462   0.63577
  0.34039  -0.73039   0.21134   0.018396 -0.30182   0.084667  0.042268
  0.14942   0.69242 

In [4]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb

In [5]:
train = pd.read_excel('QA_from_val.xlsx')
train.head()

,Unnamed: 0,Logeq,Mutex,Necc,answer,qid,question
0,0,is this dude using knife to cut the cake?,is this dude using a hammer to cut the cake?,is there any cake in the picture?,knife,340069003,This bored dude is using what instrument to cu...
1,3,is a city bus pictured?,is the type of bus pictured a tourist bus?,is there a bus pictured?,city bus,188817001,What type of bus is pictured?
2,8,are there logs on the ground behind the giraffe?,is there spoon laying on the ground behind the...,is there a giraffe?,logs,573778003,What is laying on the ground behind the giraffe?
3,10,is the horse brown?,is the horse black?,is there a horse?,brown,405135001,What color is the horse?
4,11,is the surfboard green?,is the surfboard yellow?,is there a surfboard?,green,187362006,What color is the surfboard?


In [12]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
mOdel = model_class.from_pretrained(pretrained_weights)

In [6]:
trainset=[]
for i in range(0,len(train['question'])):
    trainset.append(str(train['question'][i])+" "+str(train['answer'][i]))
train['input']=trainset

In [14]:
print(train['input'])

0        This bored dude is using what instrument to cu...
1                   What type of bus is pictured? city bus
2        What is laying on the ground behind the giraff...
3                           What color is the horse? brown
4                       What color is the surfboard? green
                               ...                        
10316             What is the man talking into? microphone
10317    What is in between the trees and the parking s...
10318    What is the gray, white, and black item in the...
10319                   How many birds are on the posts? 4
10320        What is the guy wearing on his back? backpack
Name: input, Length: 10321, dtype: object


In [329]:
tokenized = train['input'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [333]:
print(tokenized[0])

[101, 2023, 11471, 12043, 2003, 2478, 2054, 6602, 2000, 3013, 1996, 9850, 1029, 5442, 102]


In [16]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10321, 24)

In [18]:
input_ids = torch.tensor(padded)  
for i in range(len(input_ids)):
    for j in range(len(input_ids[i])):
        input_ids[i][j]=int(input_ids[i][j])
attention_mask = torch.tensor(attention_mask)

In [19]:
input_ids = torch.tensor(input_ids).to(torch.int64)

C:\Users\Manish\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [21]:
with torch.no_grad():
    last_hidden_states = mOdel(input_ids, attention_mask=attention_mask)

In [22]:
print(last_hidden_states[0])

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [227]:
total_length=len(last_hidden_states[0])
test_length=2*total_length//3
print(total_length)
print(test_length)

10321
6880


In [228]:
test_input=last_hidden_states[0][:test_length]
val_input=last_hidden_states[0][test_length:]
print(test_input)
print()
print(val_input)

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [25]:
test_output_sentences=train['Logeq'][:test_length]
val_output_sentences=train['Logeq'][test_length:]
print(test_output_sentences)
print(val_output_sentences)

0              is this dude using knife to cut the cake?
1                                is a city bus pictured?
2       are there logs on the ground behind the giraffe?
3                                    is the horse brown?
4                                is the surfboard green?
                              ...                       
6875                              are there 12 cupcakes?
6876                       is she wearing a white shirt?
6877                    is that heineken  brand of beer?
6878                  was this photo taken in starbucks?
6879                                are there 3 glasses?
Name: Logeq, Length: 6880, dtype: object
6880                        is the man using a skateboard?
6881              is the hydrant smiling because graffiti?
6882                     is the monitor sitting on a desk?
6883                                     is the dog black?
6884                                 are 4 horses present?
                               ...   

In [9]:
vocabulary = dict()
size = 0

In [10]:
for s in train['Logeq']:
    s = s[:-1]
    sentence = s.split()
    print(sentence)
    for w in sentence:
        if w not in vocabulary:
            vocabulary[w]=size+1
            size=size+1

['is', 'this', 'dude', 'using', 'knife', 'to', 'cut', 'the', 'cake']
['is', 'a', 'city', 'bus', 'pictured']
['are', 'there', 'logs', 'on', 'the', 'ground', 'behind', 'the', 'giraffe']
['is', 'the', 'horse', 'brown']
['is', 'the', 'surfboard', 'green']
['the', 'outfit', 'the', 'woman', 'is', 'wearing', 'called', 'clothing']
['is', 'the', "person's", 'shoes', 'brown']
['is', 'color', 'they', 'are', 'wearing', 'black']
['is', 'this', 'a', 'horse']
['is', 'this', "person's", 'coat', 'blue']
['is', 'the', 'tag', 'being', 'cut']
['is', 'the', 'bred', 'of', 'this', 'dog', 'a', 'pug']
['is', 'this', 'grape', 'kind', 'of', 'wine']
['are', 'there', 'three', 'boats', 'visible']
['is', 'the', "man's", 'cap', 'red']
['are', 'there', '50', 'passengers', 'traveling', 'in', 'this', 'train']
['is', 'the', 'baby', 'on', 'his', 'lap']
['is', 'there', 'a', 'lot', 'of', 'snow', 'there']
['is', 'this', "man's", 'skis', 'blue']
['is', 'the', 'color', 'that', 'the', 'animals', 'have', 'on', 'their', 'body', '

['are', 'there', '3', 'sofas']
['is', 'there', 'Chinese', 'on', 'the', 'boards']
['is', 'the', 'shirt', 'pink']
['are', 'there', '3', "frisbee's", 'present']
['are', 'there', 'flowers', 'on', 'the', 'blue', 'umbrella', 'near', 'the', 'right', 'side']
['is', 'the', 'motorcycle', 'black']
['in', 'the', 'motorcycle', 'Indian']
['is', 'this', 'alcohol', 'bottle']
['is', 'the', 'cell', 'phone', 'black']
['is', 'this', 'a', 'building']
['is', 'the', 'camper', 'grey']
['is', 'the', 'bird', 'sitting', 'on', 'the', 'rocks']
['is', 'this', 'a', 'boat']
['are', 'these', 'donuts']
['is', 'the', 'red', 'car', 'the', 'nicest']
['is', 'the', 'man', 'fishing']
['is', 'the', 'pattern', 'on', 'the', 'blanket', 'plaid']
['are', 'the', 'people', 'getting', 'ready', 'to', 'race']
['is', 'the', 'man', 'paddle', 'boarding']
['did', 'the', 'person', 'open', 'the', 'door']
['are', 'the', 'bad', 'sheep', 'in', 'a', 'pen']
['are', '2', 'people', 'wearing', 'pink']
['is', 'there', 'a', 'cat', 'in', 'the', 'pictur

['is', 'there', 'a', 'lunch', 'truck', 'in', 'the', 'parking', 'lot']
['is', 'the', "girl's", 'shirt', 'pink']
['is', 'the', 'wall', 'street', 'on', 'the', 'sign']
['is', 'there', 'a', 'boat', 'in', 'the', 'water']
['are', 'there', '0', 'camels', 'present']
['are', 'there', '2', 'shampoo', 'bottles', 'in', 'the', 'shower']
['is', 'there', 'a', 'road', 'on', 'the', 'cake']
['are', 'they', 'riding', 'a', 'snowboard']
['is', 'the', 'floor', 'made', 'of', 'tiles']
['are', 'there', '0', 'vans']
['are', 'there', '2', 'animals']
['is', 'the', 'animal', 'brown']
['was', 'the', 'cup', 'store', 'bought']
['was', 'this', 'picture', 'taken', 'outdoors']
['are', 'there', '3', 'slices', 'of', 'cheese', 'in', 'the', 'image']
['is', 'the', 'lady', 'walking', 'in', 'the', 'woods']
['is', 'the', 'box', 'of', 'matches', 'sitting', 'on', 'a', 'book']
['is', 'the', 'kid', 'swinging', 'a', 'bat']
['is', 'the', 'middle', 'giraffe', 'youngest']
['is', 'the', 'black', 'cat', 'the', 'fluffiest']
['are', 'there'

['are', 'there', 'lights', 'on', 'the', 'bottom', 'of', 'the', 'skateboard']
['are', 'they', 'playing', 'tennis']
['is', 'the', 'beard', 'shaped', 'as', 'a', 'square']
['is', 'the', 'vehicle', 'red']
['are', 'there', '3', 'strands', 'of', 'wire', 'visible']
['is', 'the', 'grass', 'small']
['are', 'there', 'leave', 'on', 'the', 'shelf']
['is', 'the', 'black', 'sheep', 'in', 'the', 'front']
['is', 'the', 'skateboard', 'wheel', 'white']
['is', 'the', 'boat', 'gray']
['are', 'there', '2', 'buses', 'in', 'the', 'pictur']
['are', 'there', '6', 'boards', 'in', 'the', 'back', 'of', 'the', 'bench']
['is', 'there', 'a', 'woman', 'sleeping', 'on', 'the', 'couch']
['are', 'there', '0', 'paper', 'towles', 'left', 'on', 'the', 'roll']
['is', 'the', 'bat', 'white']
['are', 'this', 'photo', 'taken', 'by', 'the', 'photographer']
['is', 'there', 'a', 'sweater', 'around', 'the', "woman's", 'neck']
['is', 'the', 'baby', 'playing', 'with', 'the', 'bear']
['is', 'the', 'train', 'white', 'and', 'blue']
['is'

['does', 'it', 'say', 'fire', 'on', 'the', 'building']
['is', 'this', 'vehicle', 'train']
['does', 'the', 'front', 'tire', 'of', 'the', 'motorcycle', 'on', 'the', 'right', 'have', '20', 'spokes']
['are', 'there', '3', 'pieces', 'of', 'technology', 'in', 'this', 'picture']
['is', 'it', '2:35']
['is', 'that', 'arabic']
['is', 'the', 'tie', 'made', 'from', 'silk']
['are', 'there', '6', 'motorcycles']
['is', 'the', 'guy', 'closest', 'to', 'the', 'camera', 'eating', 'a', 'hot', 'dog']
['are', 'both', 'real', 'individuals']
['is', 'the', 'sculpture', 'around', 'the', 'clock', 'of', 'roman', 'style']
['does', 'this', 'cake', 'look', 'like', 'a', 'dog']
['is', 'the', 'teapot', 'white']
['are', 'the', 'couple', 'in', 'the', 'living', 'room']
['is', 'plane', 'in', 'the', 'air']
['is', 'the', 'microwave', 'located', 'in', 'the', 'kitchen', 'in', 'this', 'scene']
['is', 'the', 'cat', 'on', 'the', 'floor']
['is', 'chair', 'in', 'front', 'of', 'the', 'couch']
['is', 'there', '1', 'pillow']
['are', '

...

['is', 'boy', 'on', 'the', 'surfboard']
['are', '3', 'men', 'in', 'this', 'photo']
['is', 'there', '1', 'child']
['is', 'the', 'red', 'fruit', 'apple']
['are', 'apple', 'brand', 'of', 'computers', 'shown']
['are', '2', 'trains', 'there']
['are', 'stuffed', 'animals', 'piled', 'next', 'to', 'the', 'person']
['is', 'that', 'a', 'delta', 'brand', 'of', 'suitcase']
['is', 'there', 'a', 'yellow', 'line', 'on', 'the', 'ground', 'for', 'caution']
['is', 'the', 'photo', 'color']
['are', 'there', '16', 'oranges', 'in', 'the', 'picture']
['is', 'the', 'fence', 'behind']
['is', 'there', 'french', 'written', 'on', 'the', 'sign']
['is', 'there', 'a', 'frisbee', 'in', 'her', 'right', 'hand']
['are', 'there', 'rocks', 'under', 'the', 'bench']
['is', 'the', 'bear', 'brown']
['is', 'the', 'cat', 'calico']
['are', 'they', 'both', 'wearing', 'orange']
['is', 'the', 'cat', 'on', 'the', 'couch']
['is', 'there', 'a', 'beer', 'logo', 'on', 'the', 'glass']
['is', 'this', 'body', 'of', 'water', 'a', 'harbor']


['is', 'the', 'number', 'shown', '7']
['is', 'there', 'a', 'rose', 'on', 'the', 'banner']
['is', 'the', 'number', '14', 'on', 'the', 'sign']
['are', 'the', 'people', 'holding', 'umbrellas', 'because', 'it', 'is', 'sunny']
['are', 'there', 'horses', 'represented', 'in', 'the', 'maroon', 'silhouette']
['is', 'this', 'room', 'the', 'bathroom']
['are', 'the', "clock's", 'hand', 'gold']
['is', 'there', 'a', 'bench', 'in', 'the', 'park']
['are', 'the', 'pictures', 'above', 'the', 'tv', 'called', 'family', 'pictures']
['is', 'the', 'snow', 'white']
['are', 'thier', 'tongues', 'out', 'because', 'they', 'are', 'happy']
['are', 'the', "man's", 'glasses', 'clear']
['are', 'there', 'hotdogs', 'in', 'the', 'bread']
['does', 'the', 'man', 'have', 'shoes', 'on', 'his', 'feet']
['is', 'it', 'summer', 'season']
['does', 'the', 'cake', 'say', 'happy', 'birthday']
['is', 'this', 'appliance', 'a', 'micro', 'wave']
['is', 'a', 'woman', 'feeding', 'the', 'giraffe']
['are', 'the', "man's", 'shorts', 'white']

['are', '11', 'people', 'sitting', 'on', 'the', 'bench']
['are', 'the', 'horses', 'traveling', 'left']
['is', 'the', 'racket', 'nike']
['is', 'the', 'man', 'sitting', 'on', 'the', 'bench']
['are', 'there', '4', 'mountain', 'peaks']
['is', 'it', 'noon']
['is', 'there', 'snow', 'on', 'the', 'ground']
['are', 'there', 'people', 'on', 'the', 'boat']
['is', 'the', 'cat', 'laying', 'on', 'the', 'picnic', 'table']
['is', 'this', 'a', 'plane', 'runway']
['are', 'there', '5', 'horses']
['are', 'the', 'stripe', 'on', 'the', 'side', 'of', 'the', 'train', 'blue']
['are', 'there', '6', 'people', 'in', 'this', 'photo']
['are', 'there', '2', 'women', 'in', 'this', 'photo']
['does', 'the', 'sign', 'say', 'stop']
['is', 'the', 'camera', 'pointing', 'at', 'the', 'mirror']
['is', 'he', 'playing', 'tennis']
['are', 'there', '0', 'men', 'on', 'the', 'motorcycle']
['is', 'there', 'air', 'in', 'the', "umpire's", 'hand']
['was', 'this', 'picture', 'taken', 'in', 'the', 'evening']
['are', 'the', 'diamond', 'ti

['is', 'the', 'bus', 'number', '51']
['Is', 'the', 'car', 'parked']
['is', 'there', '1', 'car', 'there']
['is', 'the', 'appearance', 'of', 'the', 'city', 'scene', 'dusky']
['is', 'sticker', 'on', 'the', 'window']
['can', 'you', 'see', 'sprite', 'soda']
['does', 'she', 'have', 'one', 'hand', 'in', 'the', 'air']
['does', 'the', 'clock', 'say', '1:50']
['are', 'the', 'cows', 'on', 'the', 'farm']
['is', 'siamese', 'cat', 'in', 'this', 'luggage']
['are', '16', 'bananas', 'in', 'the', 'picture']
['is', 'there', '1', 'traffic', 'cone']
['is', 'the', 'front', 'part', 'of', 'the', 'train', 'yellow']
['is', 'the', 'sign', 'red']
['is', 'the', 'umbrella', 'blue']
['is', '1', 'method', 'of', 'transportation', 'shown']
['is', 'the', 'cat', 'on', 'the', 'dresser']
['is', 'it', 'unlikely', 'the', 'float', 'has', 'alcohol']
['are', 'the', 'beds', 'in', 'the', 'bedroom']
['is', 'the', 'color', 'of', 'the', 'grass', 'green']
['can', 'you', 'clearly', 'make', 'out', '2', 'beers']
['is', 'the', 'hydrant',

['have', '4', 'spoons', 'been', 'attached', 'to', 'the', 'toaster']
['are', 'the', 'flowers', 'in', 'this', 'picture', 'red', 'and', 'yellow']
['is', 'the', 'fifth', 'horse', 'brown', 'and', 'white']
['is', 'the', "man's", 'shirt', 'yellow']
['is', 'a', 'microwave', 'hot']
['is', 'red', 'the', 'most', 'prominent', 'color', 'on', 'the', 'train']
['are', '2', 'cats', 'here']
['are', '0', 'women', 'seated', 'on', 'the', 'bench']
['is', 'this', 'a', 'bear']
['is', 'a', 'vent', 'on', 'the', 'ceiling', 'slightly', 'over', 'the', 'toilet']
['is', 'the', 'board', 'of', 'the', 'brand', 'dc']
['are', 'the', 'scissors', 'black']
['are', '0', 'Almira', 'in', 'this', 'bathroom']
['is', 'a', 'dog', 'catching', 'the', 'Frisbee']
['Is', 'the', 'calf', 'male']
['is', 'white', 'rice', 'on', 'the', 'plate']
['is', 'this', 'building', 'made', 'of', 'brick']
['are', 'there', '6', 'yellow', 'signs']
['would', 'this', 'meal', 'eaten', 'in', 'the', 'morning']
['is', 'water', 'in', 'the', 'toilet']
['is', 'the

In [11]:
print(vocabulary)

{'is': 1, 'this': 2, 'dude': 3, 'using': 4, 'knife': 5, 'to': 6, 'cut': 7, 'the': 8, 'cake': 9, 'a': 10, 'city': 11, 'bus': 12, 'pictured': 13, 'are': 14, 'there': 15, 'logs': 16, 'on': 17, 'ground': 18, 'behind': 19, 'giraffe': 20, 'horse': 21, 'brown': 22, 'surfboard': 23, 'green': 24, 'outfit': 25, 'woman': 26, 'wearing': 27, 'called': 28, 'clothing': 29, "person's": 30, 'shoes': 31, 'color': 32, 'they': 33, 'black': 34, 'coat': 35, 'blue': 36, 'tag': 37, 'being': 38, 'bred': 39, 'of': 40, 'dog': 41, 'pug': 42, 'grape': 43, 'kind': 44, 'wine': 45, 'three': 46, 'boats': 47, 'visible': 48, "man's": 49, 'cap': 50, 'red': 51, '50': 52, 'passengers': 53, 'traveling': 54, 'in': 55, 'train': 56, 'baby': 57, 'his': 58, 'lap': 59, 'lot': 60, 'snow': 61, 'skis': 62, 'that': 63, 'animals': 64, 'have': 65, 'their': 66, 'body': 67, 'and': 68, 'white': 69, 'trees': 70, 'books': 71, 'displayed': 72, 'rack': 73, 'next': 74, 'bike': 75, 'girl': 76, 'playing': 77, 'frisbee': 78, 'window': 79, 'monito

In [12]:
vocabulary["<start>"]=size+1
size=size+1
vocabulary["<end>"]=size+1
size=size+1

In [13]:
print(size)
print(vocabulary)

3534
{'is': 1, 'this': 2, 'dude': 3, 'using': 4, 'knife': 5, 'to': 6, 'cut': 7, 'the': 8, 'cake': 9, 'a': 10, 'city': 11, 'bus': 12, 'pictured': 13, 'are': 14, 'there': 15, 'logs': 16, 'on': 17, 'ground': 18, 'behind': 19, 'giraffe': 20, 'horse': 21, 'brown': 22, 'surfboard': 23, 'green': 24, 'outfit': 25, 'woman': 26, 'wearing': 27, 'called': 28, 'clothing': 29, "person's": 30, 'shoes': 31, 'color': 32, 'they': 33, 'black': 34, 'coat': 35, 'blue': 36, 'tag': 37, 'being': 38, 'bred': 39, 'of': 40, 'dog': 41, 'pug': 42, 'grape': 43, 'kind': 44, 'wine': 45, 'three': 46, 'boats': 47, 'visible': 48, "man's": 49, 'cap': 50, 'red': 51, '50': 52, 'passengers': 53, 'traveling': 54, 'in': 55, 'train': 56, 'baby': 57, 'his': 58, 'lap': 59, 'lot': 60, 'snow': 61, 'skis': 62, 'that': 63, 'animals': 64, 'have': 65, 'their': 66, 'body': 67, 'and': 68, 'white': 69, 'trees': 70, 'books': 71, 'displayed': 72, 'rack': 73, 'next': 74, 'bike': 75, 'girl': 76, 'playing': 77, 'frisbee': 78, 'window': 79, 'm

In [14]:
for key in vocabulary:
    if key not in word_vectors:
        print(key)

person's
man's
cat's
England
Is
TV
lady's
it's
woman's
on,
North
Oxford
girl's
35's
cow's
foreground's
people's
driver's
right's
Chicago
duck's
Stop
calf's
police's
dog's
tennis?
Beach
Pizza
visilab
tree's
Zebras
men's
TVs
BOSCO
Of
photo,is
child's
USA
boys'
Frisbee
BMW
guy's
long,
morning,
Philadelphia
French
Chinese
frisbee's
Indian
boy's
bear's
Mercedes
gentleman's
Jones
Are
skatebord
don't
mcdonald's
catcher's
plane's
bread?
women's
Jersey
tshirt
sandwic
horse's
kid's
can't
girls'
tee-shirts
usa's
Do
player's
round?
baby's
Amtrak
cold,
dj's
imag
trafffic
rider's
New
York
balcony's
pictur
fluffiest
pig's
street,
According
umbrella,
row,
shelf,
bathroom?
During
America
animal's
Will
green?
Mary
Poppins
activity,
zoomed-in
English
Giraffe's
African
angel's
umpire's
selfie
tree?
batter's
snowboar
waterbottle
toiletpaper
left's
SUV
frontmost
they're
athlete's
kitchen,
damage/wear
book's
snowboarder's
'do
enter'
street?
American
birthday,
What
IS
toys'
bed's
skier's
distance,
tabl
elphan

In [54]:
word_vectors[""]

KeyError: "word '' not in vocabulary"

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
train['output']=train['Logeq']
train['output']="qwertyuiop" +" "+ train['output']+" "+"poiuytrewq"

In [32]:
train['output']

0        qwertyuiop is this dude using knife to cut the...
1           qwertyuiop  is a city bus pictured? poiuytrewq
2        qwertyuiop are there logs on the ground behind...
3               qwertyuiop  is the horse brown? poiuytrewq
4           qwertyuiop  is the surfboard green? poiuytrewq
                               ...                        
10316    qwertyuiop is the man talking into a microphon...
10317    qwertyuiop is a bike in between the trees and ...
10318    qwertyuiop is the gray, white, and black item ...
10319      qwertyuiop are 4 birds on the posts? poiuytrewq
10320    qwertyuiop is the guy wearing a backpack on hi...
Name: output, Length: 10321, dtype: object

In [33]:
tokenizer = Tokenizer(oov_token="<OOV>") # Your Code Here
tokenizer.fit_on_texts(train['output'])
word_index1 = tokenizer.word_index
print(len(word_index1))


3305


In [34]:
word_index1

{'<OOV>': 1,
 'qwertyuiop': 2,
 'poiuytrewq': 3,
 'the': 4,
 'is': 5,
 'are': 6,
 'a': 7,
 'there': 8,
 'on': 9,
 'in': 10,
 'this': 11,
 'of': 12,
 'man': 13,
 'white': 14,
 '2': 15,
 '1': 16,
 'people': 17,
 'does': 18,
 'to': 19,
 'picture': 20,
 'red': 21,
 'black': 22,
 'person': 23,
 'wearing': 24,
 'blue': 25,
 'holding': 26,
 'green': 27,
 '3': 28,
 'have': 29,
 'brown': 30,
 'cat': 31,
 'his': 32,
 'dog': 33,
 'playing': 34,
 'woman': 35,
 'and': 36,
 'shirt': 37,
 'left': 38,
 'right': 39,
 'photo': 40,
 'these': 41,
 'they': 42,
 'bus': 43,
 'made': 44,
 'sign': 45,
 'yellow': 46,
 '4': 47,
 'train': 48,
 'at': 49,
 'do': 50,
 'sitting': 51,
 'table': 52,
 'water': 53,
 'pizza': 54,
 "man's": 55,
 'for': 56,
 'color': 57,
 'it': 58,
 'you': 59,
 'standing': 60,
 'hand': 61,
 'shown': 62,
 'plate': 63,
 'animal': 64,
 'clock': 65,
 '0': 66,
 'an': 67,
 'orange': 68,
 'tennis': 69,
 'animals': 70,
 'food': 71,
 'wall': 72,
 'frisbee': 73,
 'room': 74,
 'background': 75,
 'hors

In [35]:
index_to_word=dict([(i+1,char) for (i,char) in enumerate(word_index1)])

In [36]:
sequences=tokenizer.texts_to_sequences(train['output'])
sequences

[[2, 5, 11, 1964, 206, 403, 19, 421, 4, 107, 3],
 [2, 5, 7, 422, 43, 150, 3],
 [2, 6, 8, 1468, 9, 4, 92, 93, 4, 99, 3],
 [2, 5, 4, 76, 30, 3],
 [2, 5, 4, 128, 27, 3],
 [2, 4, 590, 4, 35, 5, 24, 161, 523, 3],
 [2, 5, 4, 230, 163, 30, 3],
 [2, 5, 57, 42, 6, 24, 22, 3],
 [2, 5, 11, 7, 76, 3],
 [2, 5, 11, 230, 558, 25, 3],
 [2, 5, 4, 501, 105, 421, 3],
 [2, 5, 4, 1965, 12, 11, 33, 7, 1469, 3],
 [2, 5, 11, 1966, 354, 12, 298, 3],
 [2, 6, 8, 423, 278, 102, 3],
 [2, 5, 4, 55, 698, 21, 3],
 [2, 6, 8, 328, 1470, 591, 10, 11, 48, 3],
 [2, 5, 4, 175, 9, 32, 781, 3],
 [2, 5, 8, 7, 376, 12, 181, 8, 3],
 [2, 5, 11, 55, 424, 25, 3],
 [2, 5, 4, 57, 129, 4, 70, 29, 9, 231, 699, 30, 36, 14, 3],
 [2, 5, 4, 57, 12, 4, 124, 30, 3],
 [2, 6, 524, 525, 9, 4, 700, 134, 19, 4, 121, 3],
 [2, 5, 4, 89, 34, 73, 3],
 [2, 5, 4, 176, 9, 4, 559, 14, 3],
 [2, 5, 4, 264, 355, 995, 4, 118, 12, 4, 279, 404, 3],
 [2, 5, 7, 157, 526, 3],
 [2, 5, 11, 23, 24, 7, 22, 1967, 3],
 [2, 5, 11, 527, 641, 3],
 [2, 5, 4, 57, 12, 4, 47

In [37]:
revert=tokenizer.sequences_to_texts(sequences)
revert  
train['revert']= revert
print(train['revert'])

0        qwertyuiop is this dude using knife to cut the...
1             qwertyuiop is a city bus pictured poiuytrewq
2        qwertyuiop are there logs on the ground behind...
3                 qwertyuiop is the horse brown poiuytrewq
4             qwertyuiop is the surfboard green poiuytrewq
                               ...                        
10316    qwertyuiop is the man talking into a microphon...
10317    qwertyuiop is a bike in between the trees and ...
10318    qwertyuiop is the gray white and black item in...
10319       qwertyuiop are 4 birds on the posts poiuytrewq
10320    qwertyuiop is the guy wearing a backpack on hi...
Name: revert, Length: 10321, dtype: object


In [39]:
import re

In [40]:
revert_clean=[]
for i in range(len(train['revert'])):
    w=train["revert"][i]
    #w= re.sub(r"([?.!,¿])", r" \1 ",w)
    #w = re.sub(r'[" "]+', " ", w)
    w= re.sub(r'[\']+'," ' ", w)
    #w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()
    revert_clean.append(w)

In [41]:
revert_clean

['qwertyuiop is this dude using knife to cut the cake poiuytrewq',
 'qwertyuiop is a city bus pictured poiuytrewq',
 'qwertyuiop are there logs on the ground behind the giraffe poiuytrewq',
 'qwertyuiop is the horse brown poiuytrewq',
 'qwertyuiop is the surfboard green poiuytrewq',
 'qwertyuiop the outfit the woman is wearing called clothing poiuytrewq',
 "qwertyuiop is the person ' s shoes brown poiuytrewq",
 'qwertyuiop is color they are wearing black poiuytrewq',
 'qwertyuiop is this a horse poiuytrewq',
 "qwertyuiop is this person ' s coat blue poiuytrewq",
 'qwertyuiop is the tag being cut poiuytrewq',
 'qwertyuiop is the bred of this dog a pug poiuytrewq',
 'qwertyuiop is this grape kind of wine poiuytrewq',
 'qwertyuiop are there three boats visible poiuytrewq',
 "qwertyuiop is the man ' s cap red poiuytrewq",
 'qwertyuiop are there 50 passengers traveling in this train poiuytrewq',
 'qwertyuiop is the baby on his lap poiuytrewq',
 'qwertyuiop is there a lot of snow there poiuy

In [43]:
outputs = []
for s in revert_clean:
    outputs = outputs + [s.split()]
outputs

[['qwertyuiop',
  'is',
  'this',
  'dude',
  'using',
  'knife',
  'to',
  'cut',
  'the',
  'cake',
  'poiuytrewq'],
 ['qwertyuiop', 'is', 'a', 'city', 'bus', 'pictured', 'poiuytrewq'],
 ['qwertyuiop',
  'are',
  'there',
  'logs',
  'on',
  'the',
  'ground',
  'behind',
  'the',
  'giraffe',
  'poiuytrewq'],
 ['qwertyuiop', 'is', 'the', 'horse', 'brown', 'poiuytrewq'],
 ['qwertyuiop', 'is', 'the', 'surfboard', 'green', 'poiuytrewq'],
 ['qwertyuiop',
  'the',
  'outfit',
  'the',
  'woman',
  'is',
  'wearing',
  'called',
  'clothing',
  'poiuytrewq'],
 ['qwertyuiop',
  'is',
  'the',
  'person',
  "'",
  's',
  'shoes',
  'brown',
  'poiuytrewq'],
 ['qwertyuiop',
  'is',
  'color',
  'they',
  'are',
  'wearing',
  'black',
  'poiuytrewq'],
 ['qwertyuiop', 'is', 'this', 'a', 'horse', 'poiuytrewq'],
 ['qwertyuiop',
  'is',
  'this',
  'person',
  "'",
  's',
  'coat',
  'blue',
  'poiuytrewq'],
 ['qwertyuiop', 'is', 'the', 'tag', 'being', 'cut', 'poiuytrewq'],
 ['qwertyuiop',
  'is

In [55]:
for i in range(len(outputs)):
    outputs[i][0]='.'
outputs

[['.',
  'is',
  'this',
  'dude',
  'using',
  'knife',
  'to',
  'cut',
  'the',
  'cake',
  '?'],
 ['.', 'is', 'a', 'city', 'bus', 'pictured', '?'],
 ['.',
  'are',
  'there',
  'logs',
  'on',
  'the',
  'ground',
  'behind',
  'the',
  'giraffe',
  '?'],
 ['.', 'is', 'the', 'horse', 'brown', '?'],
 ['.', 'is', 'the', 'surfboard', 'green', '?'],
 ['.',
  'the',
  'outfit',
  'the',
  'woman',
  'is',
  'wearing',
  'called',
  'clothing',
  '?'],
 ['.', 'is', 'the', 'person', "'", 's', 'shoes', 'brown', '?'],
 ['.', 'is', 'color', 'they', 'are', 'wearing', 'black', '?'],
 ['.', 'is', 'this', 'a', 'horse', '?'],
 ['.', 'is', 'this', 'person', "'", 's', 'coat', 'blue', '?'],
 ['.', 'is', 'the', 'tag', 'being', 'cut', '?'],
 ['.', 'is', 'the', 'bred', 'of', 'this', 'dog', 'a', 'pug', '?'],
 ['.', 'is', 'this', 'grape', 'kind', 'of', 'wine', '?'],
 ['.', 'are', 'there', 'three', 'boats', 'visible', '?'],
 ['.', 'is', 'the', 'man', "'", 's', 'cap', 'red', '?'],
 ['.',
  'are',
  'there'

In [56]:
vocabulary = dict()
size = 0

In [57]:
for s in outputs:
    for w in s:
        if w not in vocabulary:
            vocabulary[w] = size+1
            size = size + 1
size
vocabulary

{'.': 1,
 'is': 2,
 'this': 3,
 'dude': 4,
 'using': 5,
 'knife': 6,
 'to': 7,
 'cut': 8,
 'the': 9,
 'cake': 10,
 '?': 11,
 'a': 12,
 'city': 13,
 'bus': 14,
 'pictured': 15,
 'are': 16,
 'there': 17,
 'logs': 18,
 'on': 19,
 'ground': 20,
 'behind': 21,
 'giraffe': 22,
 'horse': 23,
 'brown': 24,
 'surfboard': 25,
 'green': 26,
 'outfit': 27,
 'woman': 28,
 'wearing': 29,
 'called': 30,
 'clothing': 31,
 'person': 32,
 "'": 33,
 's': 34,
 'shoes': 35,
 'color': 36,
 'they': 37,
 'black': 38,
 'coat': 39,
 'blue': 40,
 'tag': 41,
 'being': 42,
 'bred': 43,
 'of': 44,
 'dog': 45,
 'pug': 46,
 'grape': 47,
 'kind': 48,
 'wine': 49,
 'three': 50,
 'boats': 51,
 'visible': 52,
 'man': 53,
 'cap': 54,
 'red': 55,
 '50': 56,
 'passengers': 57,
 'traveling': 58,
 'in': 59,
 'train': 60,
 'baby': 61,
 'his': 62,
 'lap': 63,
 'lot': 64,
 'snow': 65,
 'skis': 66,
 'that': 67,
 'animals': 68,
 'have': 69,
 'their': 70,
 'body': 71,
 'and': 72,
 'white': 73,
 'trees': 74,
 'books': 75,
 'displaye

In [60]:
print(vocabulary)

{'.': 1, 'is': 2, 'this': 3, 'dude': 4, 'using': 5, 'knife': 6, 'to': 7, 'cut': 8, 'the': 9, 'cake': 10, '?': 11, 'a': 12, 'city': 13, 'bus': 14, 'pictured': 15, 'are': 16, 'there': 17, 'logs': 18, 'on': 19, 'ground': 20, 'behind': 21, 'giraffe': 22, 'horse': 23, 'brown': 24, 'surfboard': 25, 'green': 26, 'outfit': 27, 'woman': 28, 'wearing': 29, 'called': 30, 'clothing': 31, 'person': 32, "'": 33, 's': 34, 'shoes': 35, 'color': 36, 'they': 37, 'black': 38, 'coat': 39, 'blue': 40, 'tag': 41, 'being': 42, 'bred': 43, 'of': 44, 'dog': 45, 'pug': 46, 'grape': 47, 'kind': 48, 'wine': 49, 'three': 50, 'boats': 51, 'visible': 52, 'man': 53, 'cap': 54, 'red': 55, '50': 56, 'passengers': 57, 'traveling': 58, 'in': 59, 'train': 60, 'baby': 61, 'his': 62, 'lap': 63, 'lot': 64, 'snow': 65, 'skis': 66, 'that': 67, 'animals': 68, 'have': 69, 'their': 70, 'body': 71, 'and': 72, 'white': 73, 'trees': 74, 'books': 75, 'displayed': 76, 'rack': 77, 'next': 78, 'bike': 79, 'girl': 80, 'playing': 81, 'fri

In [59]:
vocabulary[''] = 0

In [61]:
size = size + 1

In [62]:
size

3231

In [63]:
reverse_vocabulary = dict()

In [459]:
for key in vocabulary:
    reverse_vocabulary[vocabulary[key]] = key
print(reverse_vocabulary)

{1: '.', 2: 'is', 3: 'this', 4: 'dude', 5: 'using', 6: 'knife', 7: 'to', 8: 'cut', 9: 'the', 10: 'cake', 11: '?', 12: 'a', 13: 'city', 14: 'bus', 15: 'pictured', 16: 'are', 17: 'there', 18: 'logs', 19: 'on', 20: 'ground', 21: 'behind', 22: 'giraffe', 23: 'horse', 24: 'brown', 25: 'surfboard', 26: 'green', 27: 'outfit', 28: 'woman', 29: 'wearing', 30: 'called', 31: 'clothing', 32: 'person', 33: "'", 34: 's', 35: 'shoes', 36: 'color', 37: 'they', 38: 'black', 39: 'coat', 40: 'blue', 41: 'tag', 42: 'being', 43: 'bred', 44: 'of', 45: 'dog', 46: 'pug', 47: 'grape', 48: 'kind', 49: 'wine', 50: 'three', 51: 'boats', 52: 'visible', 53: 'man', 54: 'cap', 55: 'red', 56: '50', 57: 'passengers', 58: 'traveling', 59: 'in', 60: 'train', 61: 'baby', 62: 'his', 63: 'lap', 64: 'lot', 65: 'snow', 66: 'skis', 67: 'that', 68: 'animals', 69: 'have', 70: 'their', 71: 'body', 72: 'and', 73: 'white', 74: 'trees', 75: 'books', 76: 'displayed', 77: 'rack', 78: 'next', 79: 'bike', 80: 'girl', 81: 'playing', 82: 

In [66]:
answer_sequences = []
for s in outputs:
    x = []
    for w in s:
        x = x + [vocabulary[w]]
    answer_sequences = answer_sequences + [x]
print(answer_sequences)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 12, 13, 14, 15, 11], [1, 16, 17, 18, 19, 9, 20, 21, 9, 22, 11], [1, 2, 9, 23, 24, 11], [1, 2, 9, 25, 26, 11], [1, 9, 27, 9, 28, 2, 29, 30, 31, 11], [1, 2, 9, 32, 33, 34, 35, 24, 11], [1, 2, 36, 37, 16, 29, 38, 11], [1, 2, 3, 12, 23, 11], [1, 2, 3, 32, 33, 34, 39, 40, 11], [1, 2, 9, 41, 42, 8, 11], [1, 2, 9, 43, 44, 3, 45, 12, 46, 11], [1, 2, 3, 47, 48, 44, 49, 11], [1, 16, 17, 50, 51, 52, 11], [1, 2, 9, 53, 33, 34, 54, 55, 11], [1, 16, 17, 56, 57, 58, 59, 3, 60, 11], [1, 2, 9, 61, 19, 62, 63, 11], [1, 2, 17, 12, 64, 44, 65, 17, 11], [1, 2, 3, 53, 33, 34, 66, 40, 11], [1, 2, 9, 36, 67, 9, 68, 69, 19, 70, 71, 24, 72, 73, 11], [1, 2, 9, 36, 44, 9, 74, 24, 11], [1, 16, 75, 76, 19, 9, 77, 78, 7, 9, 79, 11], [1, 2, 9, 80, 81, 82, 11], [1, 2, 9, 83, 19, 9, 84, 73, 11], [1, 2, 9, 85, 86, 87, 9, 88, 44, 9, 89, 33, 34, 90, 11], [1, 2, 12, 91, 92, 11], [1, 2, 3, 32, 29, 12, 38, 93, 11], [1, 2, 3, 94, 95, 11], [1, 2, 9, 36, 44, 9, 96, 9, 53, 97, 9, 82, 

In [67]:
max = 0
for s in outputs:
    if len(s)>max:
        max = len(s)
print(max)

20


In [458]:
padded = []
for s in answer_sequences:
    padded = padded + [s + (max - len(s))*[0]]
print(padded)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 12, 13, 14, 15, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 16, 17, 18, 19, 9, 20, 21, 9, 22, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 23, 24, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 25, 26, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 9, 27, 9, 28, 2, 29, 30, 31, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 32, 33, 34, 35, 24, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 36, 37, 16, 29, 38, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 3, 12, 23, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 3, 32, 33, 34, 39, 40, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 41, 42, 8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 43, 44, 3, 45, 12, 46, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 3, 47, 48, 44, 49, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 16, 17, 50, 51, 52, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 9, 53, 33, 34, 54, 55, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [69]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [70]:
size

3231

In [104]:
input_d = []
output_d = []
for key in vocabulary:
    if key in word_vectors:
        input_d = input_d + [np.array(word_vectors[key])]
        v = np.zeros((1,size))
        v[0][vocabulary[key]] = 1
        output_d = output_d + [v]
print(len(input_d))
print(len(output_d))

3215
3215


In [105]:
input_d = np.array(input_d)
output_d = np.array(output_d)
print(input_d)
print(output_d)

[[-0.33979   0.20941   0.46348  ... -0.23394   0.47298  -0.028803]
 [-0.54264   0.41476   1.0322   ... -1.2969    0.76217   0.46349 ]
 [-0.57058   0.44183   0.70102  ... -0.66102   0.47197   0.37253 ]
 ...
 [ 0.17579  -0.42444   0.1117   ...  0.98402   0.76311  -0.84116 ]
 [-0.54074   0.2232   -0.83073  ... -0.38271  -0.31812   0.87225 ]
 [-0.048039  0.76139  -0.33192  ...  0.14532   0.1904    0.26373 ]]
[[[0. 1. 0. ... 0. 0. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 1. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 0. 1.]]]


In [110]:
output_d_f = []
for s in output_d:
    output_d_f = output_d_f + [s[0]]
output_d_f = np.array(output_d_f)
print(output_d_f.shape)

(3215, 3231)


In [111]:
print(output_d_f)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [112]:
output_d = output_d_f

In [113]:
model = keras.Sequential(
    [
        layers.Dense(input_d.shape[1], activation="relu", name="layer1"),
        layers.Dense(output_d.shape[1], activation="softmax", name="layer2"),
    ]
)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [480]:
model.trainable = True
model.fit(input_d, output_d, epochs=50, batch_size=1, verbose=2)
model.trainable = False

Epoch 1/50
3215/3215 - 5s - loss: 0.0036
Epoch 2/50
3215/3215 - 5s - loss: 0.0044
Epoch 3/50
3215/3215 - 5s - loss: 0.0071
Epoch 4/50
3215/3215 - 5s - loss: 0.0038
Epoch 5/50
3215/3215 - 5s - loss: 2.5304e-06
Epoch 6/50
3215/3215 - 5s - loss: 0.0012
Epoch 7/50
3215/3215 - 5s - loss: 0.0056
Epoch 8/50
3215/3215 - 5s - loss: 9.2310e-04
Epoch 9/50
3215/3215 - 5s - loss: 0.0045
Epoch 10/50
3215/3215 - 5s - loss: 0.0047
Epoch 11/50
3215/3215 - 5s - loss: 0.0034
Epoch 12/50
3215/3215 - 5s - loss: 1.2414e-05
Epoch 13/50
3215/3215 - 5s - loss: 2.1026e-06
Epoch 14/50
3215/3215 - 5s - loss: 0.0016
Epoch 15/50
3215/3215 - 5s - loss: 0.0026
Epoch 16/50
3215/3215 - 5s - loss: 0.0010
Epoch 17/50
3215/3215 - 5s - loss: 0.0056
Epoch 18/50
3215/3215 - 5s - loss: 0.0033
Epoch 19/50
3215/3215 - 7s - loss: 0.0054
Epoch 20/50
3215/3215 - 5s - loss: 1.3995e-04
Epoch 21/50
3215/3215 - 5s - loss: 2.8519e-07
Epoch 22/50
3215/3215 - 5s - loss: 1.4146e-06
Epoch 23/50
3215/3215 - 5s - loss: 0.0051
Epoch 24/50
321

In [255]:
for key in vocabulary:
    key1 = key
    if key not in word_vectors:
        key1 = "cisco"
    x=reverse_vocabulary[np.argmax(model.predict(np.array([word_vectors[key1]])))]
    if (x!=key1):
        print(x)
        print(key)
        print("-------------------------------")

dell
visilab
-------------------------------
dell
skatebord
-------------------------------
dell
tshirt
-------------------------------
dell
trafffic
-------------------------------
dell
fluffiest
-------------------------------
dell
selfie
-------------------------------
dell
waterbottle
-------------------------------
dell
toiletpaper
-------------------------------
dell
frontmost
-------------------------------
dell
elphant
-------------------------------
dell
pepperonis
-------------------------------
dell
cardboards
-------------------------------
dell
yellowist
-------------------------------
dell
carraige
-------------------------------
dell
outflung
-------------------------------
dell

-------------------------------


In [256]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
mOdel = model_class.from_pretrained(pretrained_weights)

In [257]:
trainset=[]
for i in range(0,len(train['question'])):
    trainset.append(str(train['question'][i])+" "+str(train['answer'][i]))
train['input']=trainset

In [258]:
print(train['input'])

0        This bored dude is using what instrument to cu...
1                   What type of bus is pictured? city bus
2        What is laying on the ground behind the giraff...
3                           What color is the horse? brown
4                       What color is the surfboard? green
                               ...                        
10316             What is the man talking into? microphone
10317    What is in between the trees and the parking s...
10318    What is the gray, white, and black item in the...
10319                   How many birds are on the posts? 4
10320        What is the guy wearing on his back? backpack
Name: input, Length: 10321, dtype: object


In [259]:
tokenized = train['input'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [260]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [261]:
attention_mask = np.where(padded1 != 0, 1, 0)
attention_mask.shape

(10321, 24)

In [262]:
input_ids = torch.tensor(padded1)  
for i in range(len(input_ids)):
    for j in range(len(input_ids[i])):
        input_ids[i][j]=int(input_ids[i][j])
attention_mask = torch.tensor(attention_mask)

In [263]:
input_ids = torch.tensor(input_ids).to(torch.int64)

C:\Users\Manish\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [264]:
with torch.no_grad():
    last_hidden_states = mOdel(input_ids, attention_mask=attention_mask)

In [265]:
print(last_hidden_states[0])

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [266]:
total_length=len(last_hidden_states[0])
test_length=2*total_length//3
print(total_length)
print(test_length)

10321
6880


In [267]:
test_input=last_hidden_states[0][:test_length]
val_input=last_hidden_states[0][test_length:]
print(test_input)
print()
print(val_input)

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [268]:
test_output_sentences=train['Logeq'][:test_length]
val_output_sentences=train['Logeq'][test_length:]
print(test_output_sentences)
print(val_output_sentences)

0              is this dude using knife to cut the cake?
1                                is a city bus pictured?
2       are there logs on the ground behind the giraffe?
3                                    is the horse brown?
4                                is the surfboard green?
                              ...                       
6875                              are there 12 cupcakes?
6876                       is she wearing a white shirt?
6877                    is that heineken  brand of beer?
6878                  was this photo taken in starbucks?
6879                                are there 3 glasses?
Name: Logeq, Length: 6880, dtype: object
6880                        is the man using a skateboard?
6881              is the hydrant smiling because graffiti?
6882                     is the monitor sitting on a desk?
6883                                     is the dog black?
6884                                 are 4 horses present?
                               ...   

In [269]:
max_len = len(padded[0])
max_len

20

In [457]:
print(padded[3])

[1, 2, 9, 23, 24, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [271]:
total_words = size

In [452]:
decoder_input_data = np.zeros(
    (len(padded), max_len, total_words),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(padded), max_len, total_words),
    dtype='float32')

batch_size = 64
epochs = 100
latent_dim = 200
num_samples = test_input.shape[0]

print(test_input.shape)

torch.Size([6880, 24, 768])


In [273]:
from __future__ import print_function
import numpy as np

In [274]:
size

3231

In [275]:
from __future__ import print_function

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [453]:
for i, seq in enumerate(padded):
    for t, num in enumerate(seq):
       
        decoder_input_data[i, t, num] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, num] = 1.

In [454]:
latent_dim = 1000

In [415]:
encoder_inputs = Input(shape=(None, test_input.shape[2]))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [455]:
print(decoder_input_data)

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [456]:
print(decoder_target_data)

[[[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [379]:
decoder_input_data.shape

(6880, 20, 100)

In [460]:
decoder_input_data_new = []
for s in decoder_input_data:
    x = []
    for w in s:
        y = np.argmax(w)
        if (y==0):
            x = x + [np.zeros((1,100))[0]]
        else:
            if reverse_vocabulary[y] in word_vectors:
                x = x + [word_vectors[reverse_vocabulary[y]]]
            else:
                x = x + [word_vectors["cisco"]]
    x = np.array(x)
    decoder_input_data_new = decoder_input_data_new + [x]
decoder_input_data_new = np.array(decoder_input_data_new)
print(decoder_input_data_new)

[[[-0.33978999  0.20941     0.46348    ... -0.23394001  0.47297999
   -0.028803  ]
  [-0.54263997  0.41475999  1.03219998 ... -1.29690003  0.76217002
    0.46349001]
  [-0.57058001  0.44183001  0.70102    ... -0.66101998  0.47196999
    0.37253001]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.33978999  0.20941     0.46348    ... -0.23394001  0.47297999
   -0.028803  ]
  [-0.54263997  0.41475999  1.03219998 ... -1.29690003  0.76217002
    0.46349001]
  [-0.27085999  0.044006   -0.02026    ... -0.4923      0.63687003
    0.23642001]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.33978999  0.20941     0.46348 

In [461]:
decoder_input_data_new.shape

(10321, 20, 100)

In [462]:
decoder_input_data = decoder_input_data_new

In [463]:
print(decoder_target_data)

[[[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [464]:
decoder_input_data=decoder_input_data[:test_length]

In [465]:
decoder_target_data=decoder_target_data[:test_length]

In [466]:
encoder_input_data=test_input

In [467]:
print(encoder_input_data.shape)

torch.Size([6880, 24, 768])


In [468]:
model.trainable = False

In [469]:
decoder_inputs = Input(shape=(None, decoder_input_data.shape[2]))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_layer = Dense(500, activation="relu", name="layerx")
decoder_outputs = decoder_layer(decoder_outputs)
decoder_layer2 = Dense(100, activation="relu", name="layery")
decoder_outputs = decoder_layer2(decoder_outputs)
decoder_outputs = model(decoder_outputs, training = False)

final_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [470]:
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)
decoder_target_data = np.array(decoder_target_data)

In [472]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [474]:
final_model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=50,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
111/111 [==============================] - 133s 1s/step - loss: 2.5507 - accuracy: 0.6676 - val_loss: 1.6190 - val_accuracy: 0.7932
Epoch 2/50
111/111 [==============================] - 132s 1s/step - loss: 1.3517 - accuracy: 0.8148 - val_loss: 1.4427 - val_accuracy: 0.8137
Epoch 3/50
111/111 [==============================] - 137s 1s/step - loss: 1.1050 - accuracy: 0.8396 - val_loss: 1.3041 - val_accuracy: 0.8253
Epoch 4/50
111/111 [==============================] - 134s 1s/step - loss: 0.9219 - accuracy: 0.8578 - val_loss: 1.2458 - val_accuracy: 0.8332
Epoch 5/50
111/111 [==============================] - 131s 1s/step - loss: 0.7761 - accuracy: 0.8745 - val_loss: 1.2245 - val_accuracy: 0.8403
Epoch 6/50
111/111 [==============================] - 132s 1s/step - loss: 0.6687 - accuracy: 0.8873 - val_loss: 1.2199 - val_accuracy: 0.8455
Epoch 7/50
111/111 [==============================] - 131s 1s/step - loss: 0.5756 - accuracy: 0.8994 - val_loss: 1.1841 - val_accuracy: 0.8483

In [475]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,)) 
decoder_state_input_c = Input(shape=(latent_dim,)) 
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] 
decoder_outputs, state_h, state_c = decoder_lstm( decoder_inputs, initial_state=decoder_states_inputs) 
decoder_states = [state_h, state_c] 
decoder_outputs = decoder_layer(decoder_outputs) 
decoder_outputs = decoder_layer2(decoder_outputs) 
decoder_outputs = model(decoder_outputs) 
decoder_model = Model( [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [476]:
max_len

20

In [477]:
def decode_sequence(input_seq):
    count=0
    states_value = encoder_model.predict(input_seq)
    

    target_seq = [[word_vectors["."]]]
    target_seq = np.array(target_seq)
    count += 1

    stop_condition = False
    decoded_sentence = '. '
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        count += 1

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_vocabulary[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_sentence += ' '

        if (sampled_token_index == 11 or
           count > max_len or sampled_token_index == 0):
            stop_condition = True
            break

        target_seq = np.array([[word_vectors[sampled_char]]])

        states_value = [h, c]

    return decoded_sentence

In [478]:
for seq_index in range(100):
    ipt=train['question'][6880+seq_index+500]
    ts=val_input[seq_index+500:seq_index+1+500]
    ts = np.array(ts)
    dcs=decode_sequence(ts)
    print('Input sentence:', ipt)
    print('Answer:', train['answer'][6880+seq_index+500])
    print('Decoded sentence:', dcs)

Input sentence: What number is on the fielders uniform?
Answer: 24
Decoded sentence: . is the number on the batter 70 appeared ? 
Input sentence: What type of scene is this?
Answer: safari
Decoded sentence: . is this against no film crock ? 
Input sentence: What is the chair made out of?
Answer: wood
Decoded sentence: . is the stuff made of wood ? 
Input sentence: What type of vehicle is this?
Answer: train
Decoded sentence: . is this a train ? 
Input sentence: What are they drinking with their sandwich?
Answer: diet coke
Decoded sentence: . are they ate citizen like loading wine ? 
Input sentence: How many umbrellas are in this picture?
Answer: 1
Decoded sentence: . is there one umbrella in this picture ? 
Input sentence: What color is the wall?
Answer: white
Decoded sentence: . is the wall white ? 
Input sentence: How many boats are in the picture?
Answer: 1
Decoded sentence: . is there 1 boat in the picture ? 
Input sentence: How many wheels are on the skateboard?
Answer: 4
Decoded 

Input sentence: What company manufactured the blue car?
Answer: volkswagen
Decoded sentence: . is the maker of the soldier 3rd tire ? 
Input sentence: Which sport item is higher in the air?
Answer: frisbee
Decoded sentence: . is restaurant having taking iphone visible ? 
Input sentence: What color is the sink?
Answer: black
Decoded sentence: . is the sink black ? 
Input sentence: What is the girl holding?
Answer: nothing
Decoded sentence: . is the girl holding nothing ? 
Input sentence: What is the cake made to look like?
Answer: paw
Decoded sentence: . is the hat most circles to a packing ? 
Input sentence: What are the people doing tonight?
Answer: prom
Decoded sentence: . are the people posing at one closest ? 
Input sentence: Who are these men?
Answer: speakers
Decoded sentence: . are these men airport ? 
Input sentence: What is in the blender?
Answer: food
Decoded sentence: . is ? 
Input sentence: What color is the sky?
Answer: blue
Decoded sentence: . is the sky blue ? 
Input sen

In [479]:
for seq_index in range(100):
    ipt=train['question'][seq_index+500]
    ts=test_input[seq_index+500:seq_index+1+500]
    ts = np.array(ts)
    dcs=decode_sequence(ts)
    print('Input sentence:', ipt)
    print('Answer:', train['answer'][seq_index+500])
    print('Decoded sentence:', dcs)

Input sentence: How many remotes can be seen?
Answer: 2
Decoded sentence: . can 2 remotes be seen ? 
Input sentence: What is the action reflected in the mirror?
Answer: taking picture
Decoded sentence: . is the picture taking action reflected in the mirror ? 
Input sentence: How would you describe the zoning of this area?
Answer: residential
Decoded sentence: . would you describe the zoning of this area growing ? 
Input sentence: What is this object used for?
Answer: wii
Decoded sentence: . is this object used for wii ? 
Input sentence: What is the lady kissing?
Answer: cat
Decoded sentence: . is the lady kissing the cat ? 
Input sentence: What kind of animals are walking towards the left of the screen?
Answer: zebras
Decoded sentence: . are the zebras walking towards the left of the screen ? 
Input sentence: What kind of tree is in the background?
Answer: oak
Decoded sentence: . is there an oak tree in the background ? 
Input sentence: What is the boy holding?
Answer: baseball bat
Dec

Input sentence: How many things are hanging on the refrigerator?
Answer: 11
Decoded sentence: . are there 11 things hanging on the refrigerator ? 
Input sentence: Which cow is all black?
Answer: back
Decoded sentence: . is the back cow all black ? 
Input sentence: What is in the pockets?
Answer: banana
Decoded sentence: . is there a banana in the pockets ? 
Input sentence: What team does the man play for?
Answer: orioles
Decoded sentence: . does the man play preparing orioles ? 
Input sentence: What game is being played?
Answer: tennis
Decoded sentence: . is tennis being played ? 
Input sentence: What color is the chair cushion?
Answer: red
Decoded sentence: . is the chair cushion red ? 
Input sentence: What fruit can be seen?
Answer: tomato
Decoded sentence: . is there a fruit ? 
Input sentence: What is above the gages?
Answer: bow
Decoded sentence: . is there a bow above the gages ? 
Input sentence: Which teams dugout is shown, visitors or home?
Answer: home
Decoded sentence: . is th

In [426]:
input_new_sentence = pd.read_excel("inputs.xlsx")

In [427]:
input_new_sentence["input"] = ["what is this animal? cat"]

In [428]:
tokenized = input_new_sentence['input'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [429]:
print(tokenized)

0    [101, 2054, 2003, 2023, 4111, 1029, 4937, 102]
Name: input, dtype: object


In [430]:
padded_new = np.array([i + [0]*(24-len(i)) for i in tokenized.values])

In [431]:
attention_mask = np.where(padded_new != 0, 1, 0)
attention_mask.shape

(1, 24)

In [432]:
input_ids = torch.tensor(padded_new)  
for i in range(len(input_ids)):
    for j in range(len(input_ids[i])):
        input_ids[i][j]=int(input_ids[i][j])
attention_mask = torch.tensor(attention_mask)

In [433]:
input_ids = torch.tensor(input_ids).to(torch.int64)

C:\Users\Manish\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [434]:
print(input_ids)

tensor([[ 101, 2054, 2003, 2023, 4111, 1029, 4937,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])


In [435]:
with torch.no_grad():
    last_hidden_states = mOdel(input_ids, attention_mask=attention_mask)

In [436]:
print(last_hidden_states[0])

tensor([[[-0.1539, -0.0408, -0.0871,  ..., -0.1947,  0.5742,  0.3938],
         [-0.0417, -0.0720, -0.3726,  ...,  0.1312,  0.6042, -0.0609],
         [-0.2278, -0.0154, -0.1685,  ..., -0.2982,  0.4036,  0.6726],
         ...,
         [ 0.0746,  0.0339,  0.0471,  ...,  0.0417,  0.1809,  0.3003],
         [ 0.0973,  0.0283, -0.0655,  ...,  0.0170,  0.1780,  0.2893],
         [ 0.0222,  0.0612, -0.0571,  ..., -0.0120,  0.1921,  0.2924]]])


In [437]:
print('Input sentence:', input_new_sentence["input"][0])
print('Decoded sentence:', decode_sequence(np.array(last_hidden_states[0])))
dcs=decode_sequence(ts)

Input sentence: what is this animal? cat
Decoded sentence: . it a cat ? 


In [438]:
def find_logec_question(input_s):
    input_new_sentence["input"] = [input_s]
    tokenized = input_new_sentence['input'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    padded_new = np.array([i + [0]*(24-len(i)) for i in tokenized.values])
    attention_mask = np.where(padded_new != 0, 1, 0)
    input_ids = torch.tensor(padded_new)  
    for i in range(len(input_ids)):
        for j in range(len(input_ids[i])):
            input_ids[i][j]=int(input_ids[i][j])
    attention_mask = torch.tensor(attention_mask)
    input_ids = torch.tensor(input_ids).to(torch.int64)
    with torch.no_grad():
        last_hidden_states = mOdel(input_ids, attention_mask=attention_mask)
    print('Input sentence:', input_new_sentence["input"][0])
    print('Decoded sentence:', decode_sequence(np.array(last_hidden_states[0])))
    print("------------------------------------------")

In [451]:
find_logec_question("What are they playing? baseball")

C:\Users\Manish\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


Input sentence: What are they playing? baseball
Decoded sentence: . young sure sports baseball ? 
------------------------------------------
